# (baseline) Most Popular Articles split by age group

In [1]:
import os
os.chdir('/home/jovyan/work')

In [2]:
import polars as pl

In [3]:
import notebooks
from notebooks import data_storage

transactions = data_storage.load_dataset('transactions_train')
customers = data_storage.load_dataset('customers')

importing Jupyter notebook from /home/jovyan/work/notebooks/data_storage.ipynb


In [5]:
class MostPopularByAgeGroupRecommender:
  def __init__(self, transactions: pl.DataFrame):
    self.transactions = transactions
    self.customers = customers

  def get_recommendations(self, customer_ids: list, start_date, end_date) -> pl.DataFrame:
    # TODO generate most_popular per-age-group
    most_popular = (
      transactions
      .lazy()
      .filter(
        (pl.col('t_dat') < end_date)
      & (pl.col('t_dat') > start_date)
      )
      .groupby('article_id')
      .agg([pl.count()])
      .sort('count', reverse=True)
      .head(12)
      .collect()
    )

    predictions = " ".join(map(str, most_popular['article_id'].to_list()))

    df = pl.DataFrame({
      'customer_id': customer_ids,
      'prediction': [predictions] * len(customer_ids),
    })

    return df

In [6]:
sample_submission = pl.read_csv('data/sample_submission.csv')

In [7]:
transactions.select([pl.max('t_dat').alias('max_date'), pl.min('t_dat').alias('min_date')])

max_date,min_date
str,str
null,null


In [ ]:
rec = MostPopularByAgeGroup(transactions)
submission = rec.get_recommendations(sample_submission['customer_id'], '2018-09-20', '2020-09-23')

In [ ]:
submission.write_csv('most-popular-by-age-group-submission.csv')